In [56]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

data=fetch_california_housing()
df=pd.DataFrame(data.data,columns=data.feature_names)
df['MedHouseVal']=data.target
df.sample(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
9841,2.4663,39.0,4.220207,1.095855,804.0,2.082902,36.60,-121.90,2.365
6826,3.8906,46.0,5.426923,1.009615,1434.0,2.757692,34.09,-118.10,2.383
14213,3.6250,21.0,4.881773,1.034483,695.0,3.423645,32.69,-117.05,1.443
17673,2.6763,16.0,3.953010,1.099853,1674.0,2.458150,37.30,-121.88,1.911
15720,3.7159,52.0,4.574830,0.989796,560.0,1.904762,37.78,-122.45,4.944


In [57]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X=df.drop(columns=['MedHouseVal'])

X_const=add_constant(X)

vif=pd.DataFrame()
vif['Features']=X.columns
vif['VIF']=[variance_inflation_factor(X_const.values, i+1) for i in range(X.shape[1])]
print(vif)

     Features       VIF
0      MedInc  2.501295
1    HouseAge  1.241254
2    AveRooms  8.342786
3   AveBedrms  6.994995
4  Population  1.138125
5    AveOccup  1.008324
6    Latitude  9.297624
7   Longitude  8.962263


In [58]:
def reduce_multicollinearity(X,threshold=9.0):
  while True:
    X_const = add_constant(X)
    vif=pd.Series(
        [variance_inflation_factor(X_const.values, i+1) for i in range(X.shape[1])],
        index=X.columns
    )
    max_vif =vif.max()
    if max_vif > threshold:
      feature_to_drop = vif.idxmax()
      print(f"Dropping '{feature_to_drop}' with VIF = {max_vif:.2f}")
      X=X.drop(columns=[feature_to_drop])
    else:
      break
  return X

X_reduced = reduce_multicollinearity(X)
X_reduced

Dropping 'Latitude' with VIF = 9.30


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,-122.25
...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,-121.32
